使用資料增強技術搭配正則化(regularization)技術，例如:early stopping, Bagging

In [23]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Version A：原始資料（無增強）
raw_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# Version B：訓練用資料增強
aug_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3,
                             scale=(0.02, 0.15),
                             ratio=(0.3, 3.3),
                             value=0)  # 或用 'random'
])

# 載入訓練／測試 Dataset
raw_train_full = datasets.ImageFolder(root="./data/train", transform=raw_transform)
raw_test     = datasets.ImageFolder(root="./data/test",  transform=raw_transform)

aug_train_full = datasets.ImageFolder(root="./data/train", transform=aug_transform)
#raw_transform 以評估增強後模型真實表現
aug_val_full  = datasets.ImageFolder(root="./data/train", transform=raw_transform)
aug_test      = datasets.ImageFolder(root="./data/test",  transform=raw_transform)

print(" 類別對應：", raw_train_full.classes)
print(" 訓練筆數（原始）：", len(raw_train_full))
print(" 測試筆數：", len(raw_test))

# 切分訓練 / 驗證集（8:2）
train_ratio = 0.8
raw_train_size = int(len(raw_train_full) * train_ratio)
raw_val_size   = len(raw_train_full) - raw_train_size
raw_train, raw_val = random_split(raw_train_full, [raw_train_size, raw_val_size])

aug_train_size = int(len(aug_train_full) * train_ratio)
aug_val_size   = len(aug_train_full) - aug_train_size
aug_train, _   = random_split(aug_train_full, [aug_train_size, len(aug_train_full)-aug_train_size])
_, aug_val     = random_split(aug_val_full, [aug_train_size, len(aug_val_full)-aug_train_size])

# DataLoader
batch_size = 64
raw_train_loader = DataLoader(raw_train, batch_size=batch_size, shuffle=True)
raw_val_loader   = DataLoader(raw_val,   batch_size=batch_size, shuffle=False)
raw_test_loader  = DataLoader(raw_test,  batch_size=batch_size, shuffle=False)

aug_train_loader = DataLoader(aug_train, batch_size=batch_size, shuffle=True)
aug_val_loader   = DataLoader(aug_val,   batch_size=batch_size, shuffle=False)
aug_test_loader  = DataLoader(aug_test,  batch_size=batch_size, shuffle=False)


 類別對應： ['men', 'women']
 訓練筆數（原始）： 220
 測試筆數： 80


Dropping

In [24]:
import copy
import torch
import torch.nn as nn

def train_model(model, train_loader, val_loader, num_epochs, device, patience, label=''):
   
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    best_acc = 0.0
    best_state = copy.deepcopy(model.state_dict())
    no_improve = 0

    print(f"開始訓練 with EarlyStopping(patience={patience}): {label}")
    for epoch in range(1, num_epochs + 1):
        # --- Train ---
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)               # forward 只回傳 logits
            loss = criterion(outputs, labels)     # 只用 CE Loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total

        # --- Validate ---
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        val_acc = 100 * val_correct / val_total

        print(
            f"[{label}] Epoch {epoch}/{num_epochs}  "
            f"Loss: {running_loss:.4f}  "
            f"Train Acc: {train_acc:.2f}%  Val Acc: {val_acc:.2f}%"
        )

        # --- Early Stopping Check ---
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = copy.deepcopy(model.state_dict())
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping triggered (no improvement for {patience} epochs).")
                break

    # 載入最佳權重
    model.load_state_dict(best_state)
    print(f"[{label}] 最佳驗證準確率: {best_acc:.2f}%")
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [25]:
def evaluate_model(model, test_loader, device, label=""):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"[{label}]  測試集準確率：{acc:.2f}%")
    return acc

In [26]:
import torchvision.models as models
import torch.nn as nn

class GenderCNN_Dropout(nn.Module):
    def __init__(self):
        super().__init__()
        # 載入預訓練 EfficientNet‑B0
        self.backbone = models.efficientnet_b0(pretrained=True)
        # 冻结大部分層，只解凍最後五個 block
        for param in self.backbone.parameters():
            param.requires_grad = False
        for param in self.backbone.features[-5:].parameters():
            param.requires_grad = True

        # 替換分類頭：純線性 + ReLU
        self.backbone.classifier[1] = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(inplace=True),

            nn.Linear(512, 256),
            nn.ReLU(inplace=True),

            nn.Linear(256, 2)
        )

    def forward(self, x):
        return self.backbone(x)


In [27]:
# 訓練參數
num_epochs = 20
patience   = 7  
model_raw = GenderCNN_Dropout().to(device)
print("\n" + "="*30 + " Training Raw Model " + "="*30)
model_raw = train_model(model_raw, raw_train_loader, raw_val_loader, num_epochs, device, patience, label="raw")

model_aug = GenderCNN_Dropout().to(device)
print("\n" + "="*30 + " Training Augmented Model " + "="*30)
model_aug = train_model(model_aug, aug_train_loader, aug_val_loader, num_epochs, device, patience, label="augmented")



============================== Training Raw Model ==============================
開始訓練 with EarlyStopping(patience=7): raw
[raw] Epoch 1/20  Loss: 2.0637  Train Acc: 56.82%  Val Acc: 72.73%
[raw] Epoch 2/20  Loss: 1.9378  Train Acc: 94.32%  Val Acc: 88.64%
[raw] Epoch 3/20  Loss: 1.8004  Train Acc: 96.02%  Val Acc: 93.18%
[raw] Epoch 4/20  Loss: 1.6743  Train Acc: 97.16%  Val Acc: 90.91%
[raw] Epoch 5/20  Loss: 1.5659  Train Acc: 97.16%  Val Acc: 90.91%
[raw] Epoch 6/20  Loss: 1.4752  Train Acc: 97.73%  Val Acc: 90.91%
[raw] Epoch 7/20  Loss: 1.3997  Train Acc: 97.73%  Val Acc: 90.91%
[raw] Epoch 8/20  Loss: 1.3632  Train Acc: 97.16%  Val Acc: 90.91%
[raw] Epoch 9/20  Loss: 1.3219  Train Acc: 97.73%  Val Acc: 90.91%
[raw] Epoch 10/20  Loss: 1.3333  Train Acc: 96.59%  Val Acc: 90.91%
Early stopping triggered (no improvement for 7 epochs).
[raw] 最佳驗證準確率: 93.18%

============================== Training Augmented Model ==============================
開始訓練 with EarlyStopping(patience=7): aug

In [28]:
evaluate_model(model_raw, raw_test_loader, device, label="raw")
evaluate_model(model_aug, aug_test_loader, device, label="augmented")

[raw]  測試集準確率：77.50%
[augmented]  測試集準確率：78.75%


78.75

In [ ]:
import random
import copy
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler

def train_bagging_ensemble(base_model_cls, train_dataset, val_loader, test_loader, n_estimators=5, bootstrap_ratio=1.0, num_epochs=20, device=torch.device("cpu"), patience=5, label_prefix=''):

    all_models = []
    batch_size = val_loader.batch_size

    # 1. 訓練每個基學習器
    for i in range(n_estimators):
        # 1.1 Bootstrap sample
        n_samples = int(len(train_dataset) * bootstrap_ratio)
        indices = [random.randrange(len(train_dataset)) for _ in range(n_samples)]
        sampler = SubsetRandomSampler(indices)
        loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)

        # 1.2 建立並訓練模型
        model = base_model_cls().to(device)
        print(f"\n{'='*10} Bagging Model {i+1}/{n_estimators} {'='*10}")
        model = train_model(model, loader, val_loader, num_epochs, device, patience=patience, label=f"{label_prefix}#{i+1}")
        all_models.append(copy.deepcopy(model))

    # 2. 在測試集上做軟投票（平均 logits → argmax）
    correct, total = 0, 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        # 收集每個基學習器的 logits
        logits_stack = torch.stack([m(images) for m in all_models], dim=0)
        avg_logits = logits_stack.mean(dim=0)
        preds = avg_logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    test_acc = 100 * correct / total
    print(f"\nBagging Ensemble Test Acc: {test_acc:.2f}%")
    return all_models, test_acc

# 使用範例
from torchvision import transforms, datasets

# 假設 raw_transform 已定義
train_dataset = datasets.ImageFolder(root="./data/train", transform=raw_transform)

models, ensemble_acc = train_bagging_ensemble(
    base_model_cls=GenderCNN_Dropout,       
    train_dataset=train_dataset,
    val_loader=raw_val_loader,
    test_loader=raw_test_loader,
    n_estimators=5,
    bootstrap_ratio=0.7,
    num_epochs=20,
    device=device,
    patience=7,
    label_prefix='raw'
)


c:\Users\eb201\miniconda3\envs\ML_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\eb201\miniconda3\envs\ML_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



========== Bagging Model 1/5 ==========
開始訓練 with EarlyStopping(patience=7): raw#1
[raw#1] Epoch 1/20  Loss: 2.0686  Train Acc: 52.60%  Val Acc: 61.36%
[raw#1] Epoch 2/20  Loss: 1.9022  Train Acc: 77.92%  Val Acc: 77.27%
[raw#1] Epoch 3/20  Loss: 1.7461  Train Acc: 90.26%  Val Acc: 79.55%
[raw#1] Epoch 4/20  Loss: 1.6010  Train Acc: 94.81%  Val Acc: 84.09%
[raw#1] Epoch 5/20  Loss: 1.5003  Train Acc: 95.45%  Val Acc: 84.09%
[raw#1] Epoch 6/20  Loss: 1.4009  Train Acc: 97.40%  Val Acc: 84.09%
[raw#1] Epoch 7/20  Loss: 1.3835  Train Acc: 96.10%  Val Acc: 84.09%
[raw#1] Epoch 8/20  Loss: 1.3114  Train Acc: 97.40%  Val Acc: 81.82%
[raw#1] Epoch 9/20  Loss: 1.3092  Train Acc: 96.75%  Val Acc: 84.09%
[raw#1] Epoch 10/20  Loss: 1.2558  Train Acc: 96.75%  Val Acc: 86.36%
[raw#1] Epoch 11/20  Loss: 1.2348  Train Acc: 98.05%  Val Acc: 86.36%
[raw#1] Epoch 12/20  Loss: 1.2182  Train Acc: 97.40%  Val Acc: 86.36%
[raw#1] Epoch 13/20  Loss: 1.2183  Train Acc: 99.35%  Val Acc: 86.36%
[raw#1] Epoch 1

In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_individual_models(models, test_loader, device):
    
    accuracies = []
    for i, model in enumerate(models, start=1):
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)           # logits
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        acc = correct / total * 100
        accuracies.append(acc)
        print(f"Model #{i} Test Acc: {acc:.2f}%")
    return accuracies

# 1. 個別基學習器
individual_accuracies = evaluate_individual_models(models, raw_test_loader, device)



Model #1 Test Acc: 73.75%
Model #2 Test Acc: 73.75%
Model #3 Test Acc: 80.00%
Model #4 Test Acc: 62.50%
Model #5 Test Acc: 80.00%
